# Recipe: time-based aggregation

The following use cases are covered here:
 - Use case 1: Sum daily sales, from individual item transactions.
 - Use case 2: Sum daily sales by store, from store/product sales.
 - Use case 3: Unify sensor measurements with the same timestamp.


## Use case 1: Daily sales

Assume we have the sales log from a store, where each event represents one single item sale, with its corresponding timestamp, the price at which the product was sold, and the unit cost of the product.

In [ ]:
import pandas as pd
import temporian as tp

sales_data = pd.DataFrame(
    data=[
        ["2020-01-02 13:04", 3.0,  1.0],
        ["2020-01-02 13:04", 5.0,  2.0],  # duplicated timestamp
        ["2020-01-02 15:24", 7.0,  3.0],
        ["2020-01-03 13:45", 3.0,  1.0],
        ["2020-01-03 16:10", 7.0,  3.0],
        ["2020-01-03 17:30", 10.0, 5.0],
    ],
    columns=[
        "timestamp",
        "unit_price",
        "unit_cost",
    ],
)

sales_evset = tp.from_pandas(sales_data)
sales_evset

In [ ]:
time_span = tp.event_set(timestamps=["2020-01-01 00:00", "2020-01-07 00:00"])

one_day = tp.duration.days(1)
daily_ticks = time_span.tick(one_day)

daily_ticks

In [ ]:
moving_sum = sales_evset.moving_sum(window_length=one_day, sampling=daily_ticks)

moving_sum

In [ ]:
daily_sales = moving_sum.rename({"unit_price": "daily_revenue", "unit_cost": "daily_cost"})

daily_profit = (daily_sales["daily_revenue"] - daily_sales["daily_cost"]).rename("daily_profit")

daily_sales = tp.glue(daily_sales, daily_profit)

daily_sales


## Use case 2: Sales by store

In this case, let's assume that we've multiple stores with multiple products being sold.

To simplify the data, let's use daily sales indexed by store and product.

We want to accumulate all daily sales for each store, adding up all products.

In [ ]:
import pandas as pd
import temporian as tp


sales_data = pd.DataFrame(
    data=[
        # store 1
        ["2020-01-01", "store_1", "product_1", 300.0],
        ["2020-01-02", "store_1", "product_1", 450.0],
        ["2020-01-03", "store_1", "product_1", 600.0],
        ["2020-01-01", "store_1", "product_2", 100.0],
        ["2020-01-02", "store_1", "product_2", 250.0],
        ["2020-01-03", "store_1", "product_2", 100.0],
        # store 2
        ["2020-01-01", "store_2", "product_1", 900.0],
        ["2020-01-02", "store_2", "product_1", 750.0],
        ["2020-01-03", "store_2", "product_1", 750.0],
        ["2020-01-01", "store_2", "product_3", 20.0],
        ["2020-01-02", "store_2", "product_3", 40.0],
        ["2020-01-03", "store_2", "product_3", 30.0],
    ],
    columns=[
        "timestamp",
        "store_id",
        "product_id",
        "sales_usd",
    ],
)

# Load data indexed by store/product
sales_evset = tp.from_pandas(sales_data, indexes=["store_id", "product_id"])
sales_evset

Note that for each store/product we've the same 3 days of sales.

Now we want to calculate sales for each store, regardless of the `product_id`.

So, the first step is to remove that index.

In [ ]:
store_sales = sales_evset.drop_index("product_id")

store_sales

As you can see, now we've each timestamp duplicated, one for each product.

We want to discard the `product_id` in formation, and unify the timestamps adding up the sales.

In [ ]:
unique_days = store_sales.unique_timestamps()

store_daily_sales = store_sales["sales_usd"].moving_sum(window_length=tp.duration.days(1), sampling=unique_days)

store_daily_sales